# 1. Install and Import Dependencies

Run the following installation code only once (each) when first installing/using it

In [ ]:
# For Mac M1 (copy it and paste/run it on terminal)
conda install pytorch torchvision torchaudio -c pytorch

In [ ]:
# Clone Yolov5
!git clone https://github.com/ultralytics/yolov5

In [ ]:
cd yolov5

In [ ]:
!pip install -r requirements.txt

In [ ]:
# Install Python Firebase
!pip install python-firebase

In [ ]:
# Install Firebase Realtime Database - Firebase Admin
!pip install firebase_admin

# 2. Import PyTorch Object Detection Libraries

In [ ]:
import torch
from matplotlib import pyplot as plt
import numpy as np
import cv2

import json
from datetime import datetime

# 3. Load Model

In [ ]:
# model for Yolov5
model = torch.hub.load('ultralytics/yolov5', 'yolov5x')

In [ ]:
model

# 4. Setup and Import Firebase

### Create Firebase Realtime Database - Firebase Admin

i.) Create Firebase Account

ii.) Go to Console

iii.) Create Project

iv.) Click "Build" -> Click "Realtime Database"

v.)  Create Database ( Security rules: choose "locked mode")

vi.) Rules (change it to true)
{
  "rules": {
    ".read": true,
    ".write": true
  }
}

vi.i.) Rules (add .indexOn to rules) (optional: to access each child)

{
  "rules": {
    ".read": true,
    ".write": true,
      "video": {
        ".indexOn":["sensorId", "imageId", "name", "confidence", "datetime"]
      }
  }
}

vii.) Copy your firebase reference url to initalize firebase_admin app in the following code

Example URL: https://example-default-rtdb.firebaseio.com/

Example: firebase_admin = firebase_admin.initialize_app(cred, {'databaseURL': 'https://example-default-rtdb.firebaseio.com/'})

viii.) Go to "Project Setting" -> Click "Service accounts" -> Click "Python"

ix.) Generate new private key

x.) Copy the downloaded private key .json and paste it into your folder

xi.) Rename it to serviceAccountKey.json (or use its original name if you prefer)

xii.) Run the following code.

In [ ]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import db

# cred = credentials.Certificate("{path/to/serviceAccountKey.json}")
cred = credentials.Certificate("serviceAccountKey.json")

firebase_admin = firebase_admin.initialize_app(cred, {'databaseURL': '{copy your reference url and put it here}'})

# 5. Real-Time Object Detection

In [ ]:
cap = cv2.VideoCapture(0)
i = 0
pushed_classes = set()

while cap.isOpened():
    
    ret, frame = cap.read()
    results = model(frame)
    cv2.imshow('Security Camera', np.squeeze(results.render()))
    
    # PRINT OUT THE CAPTURED IMAGES RESULTS    
    print("Results")
    results.print()

    imageResult = results.pandas().xyxy[0]
    imageResult["sensorId"] = 1
    imageResult["imageId"] = i
    i = i + 1
    
    # Use DateTime now for each captured images    
    now = datetime.now()    
    imageResult["datetime"] = now.strftime("%Y/%m/%d %H:%M:%S")

    imgResult = imageResult[["sensorId", "imageId", "name", "confidence", "datetime"]]
    
    # CONVERT IT TO JSON
    lastResult = imgResult.to_json(orient="records")
    jsonResult = json.loads(lastResult)

    # Push only specific class name and push per image
    ref = db.reference('video/')
    for eachResult in jsonResult:
        if eachResult["name"] in ["person", "cell phone", "knife", "fork"] and eachResult["name"] not in pushed_classes:
            ref.push(eachResult)
            pushed_classes.add(eachResult["name"])

    if i % 25 == 0:
        pushed_classes.clear()
    
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

# 6.) Retreive Data from Firebase

In [ ]:
# Retreive all Images/Videos data from Firebase
ref = db.reference('video').get()
print(ref)

In [ ]:
# Save data from firebase and store/write it in new file "data.json"
import json

ref = db.reference('video')
data = ref.get()

# Encode the data as a JSON string
json_string = json.dumps(data)

# Write the JSON string to a file
with open("dataNew.json", "w") as f:
    f.write(json_string)

In [ ]:
# Retreive all Images/Videos data from Firebase with datetime order
ref = db.reference('video')
cref = ref.order_by_child('datetime').get()

for key, val in cref.items():
    print("key: ", key)
    print("val: ", val)

In [ ]:
# Retreive Images/Videos from Firebase by 'person' name (to find specific object.name)
ref = db.reference('video')
cref = ref.order_by_child('name').equal_to('person').get()

for key, val in cref.items():
    print("key: ", key)
    print("val: ", val)